In [3]:
# Compare BOMs between Odoo V14 and V18 to find mismatches

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Get environment variables for V14
JUSTFRAMEIT_ODOO_V14_URL = os.getenv('JUSTFRAMEIT_ODOO_V14_URL')
JUSTFRAMEIT_ODOO_V14_DB = os.getenv('JUSTFRAMEIT_ODOO_V14_DB')
JUSTFRAMEIT_ODOO_V14_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_V14_USERNAME')
JUSTFRAMEIT_ODOO_V14_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_V14_API_KEY')

# Get environment variables for V18
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

print("=" * 80)
print("Comparing BOMs between Odoo V14 and V18")
print("=" * 80)

# Helper function to normalize product codes for comparison
def normalize_code(code):
    """Remove dots from product code for comparison"""
    if code:
        return str(code).replace('.', '')
    return ''

try:
    # Authenticate to V14
    print("\n[V14] Authenticating...")
    common_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/common", allow_none=True)
    uid_v14 = common_v14.authenticate(JUSTFRAMEIT_ODOO_V14_DB, JUSTFRAMEIT_ODOO_V14_USERNAME, JUSTFRAMEIT_ODOO_V14_API_KEY, {})
    print(f"✅ V14 Authenticated! UID: {uid_v14}")
    
    # Authenticate to V18
    print("\n[V18] Authenticating...")
    common_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
    uid_v18 = common_v18.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
    print(f"✅ V18 Authenticated! UID: {uid_v18}\n")
    
    # Create models proxies
    models_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/object", allow_none=True)
    models_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)
    
    # Define specific category IDs to check
    target_category_ids = [4, 8, 10, 57, 82]
    #target_category_ids = [10] #for testing
    
    print(f"Checking specific categories: {target_category_ids}\n")
    
    # Get category details from V14 (single call)
    categories = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.category', 'read',
        [target_category_ids],
        {'fields': ['display_name']})
    
    print("Categories to check:")
    for cat in categories:
        print(f"  - Category ID {cat['id']}: {cat['display_name']}")
    print()
    
    # Get all product templates in these categories from V14 (single call)
    print("[V14] Fetching product templates in these categories...")
    product_template_ids = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.template', 'search',
        [[['categ_id', 'in', target_category_ids]]])
    
    print(f"Found {len(product_template_ids)} product templates\n")
    
    # Search for BOMs with is_custom_made = False in V14 (single call)
    print("[V14] Searching for BOMs with is_custom_made = False...")
    print("=" * 80)
    
    bom_ids_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'mrp.bom', 'search',
        [[['product_tmpl_id', 'in', product_template_ids], ['is_custom_made', '=', False]]])
    
    print(f"\n[V14] Found {len(bom_ids_v14)} BOMs with is_custom_made = False\n")
    
    if not bom_ids_v14:
        print("No BOMs found with is_custom_made = False in V14")
    else:
        # Get ALL BOM details from V14 in a single call
        boms_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
            'mrp.bom', 'read',
            [bom_ids_v14],
            {'fields': ['id', 'code', 'display_name', 'product_tmpl_id', 'product_id', 'is_custom_made', 'bom_line_ids']})
        
        # Get ALL BOM lines from V14 in a single call
        all_bom_line_ids_v14 = []
        for bom in boms_v14:
            if bom['bom_line_ids']:
                all_bom_line_ids_v14.extend(bom['bom_line_ids'])
        
        bom_lines_v14_dict = {}
        if all_bom_line_ids_v14:
            all_bom_lines_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'mrp.bom.line', 'read',
                [all_bom_line_ids_v14],
                {'fields': ['id', 'product_id', 'product_qty', 'bom_id']})
            
            # Get all product IDs from BOM lines to fetch product codes
            product_ids_v14 = [line['product_id'][0] for line in all_bom_lines_v14 if line['product_id']]
            
            # Fetch product codes for all products in V14
            products_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'product.product', 'read',
                [product_ids_v14],
                {'fields': ['id', 'product_code']})
            
            # Create a mapping of product_id to product_code
            product_code_map_v14 = {p['id']: p['product_code'] for p in products_v14}
            
            # Organize by bom_id for easy lookup
            for line in all_bom_lines_v14:
                bom_id = line['bom_id'][0] if line['bom_id'] else None
                if bom_id not in bom_lines_v14_dict:
                    bom_lines_v14_dict[bom_id] = []
                # Add product_code to the line data
                product_id = line['product_id'][0] if line['product_id'] else None
                line['product_code'] = product_code_map_v14.get(product_id, 'N/A')
                bom_lines_v14_dict[bom_id].append(line)
        
        # Collect all BOM IDs from V14 to search in V18 by code (reference)
        bom_ids_v14_as_strings = [str(bom['id']) for bom in boms_v14]
        
        # Search for ALL corresponding BOMs in V18 in a single call (matching V14 ID to V18 code)
        bom_ids_v18 = []
        if bom_ids_v14_as_strings:
            bom_ids_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'mrp.bom', 'search',
                [[['code', 'in', bom_ids_v14_as_strings]]])
        
        # Get ALL BOM details from V18 in a single call
        boms_v18_dict = {}
        if bom_ids_v18:
            boms_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'mrp.bom', 'read',
                [bom_ids_v18],
                {'fields': ['id', 'code', 'display_name', 'product_tmpl_id', 'product_id', 'bom_line_ids']})
            
            # Organize by code for easy lookup
            for bom in boms_v18:
                if bom['code']:
                    boms_v18_dict[bom['code']] = bom
            
            # Get ALL BOM lines from V18 in a single call
            all_bom_line_ids_v18 = []
            for bom in boms_v18:
                if bom['bom_line_ids']:
                    all_bom_line_ids_v18.extend(bom['bom_line_ids'])
            
            bom_lines_v18_dict = {}
            if all_bom_line_ids_v18:
                all_bom_lines_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'mrp.bom.line', 'read',
                    [all_bom_line_ids_v18],
                    {'fields': ['id', 'product_id', 'product_qty', 'bom_id']})
                
                # Get all product IDs from BOM lines to fetch product codes
                product_ids_v18 = [line['product_id'][0] for line in all_bom_lines_v18 if line['product_id']]
                
                # Fetch product codes for all products in V18
                products_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'product.product', 'read',
                    [product_ids_v18],
                    {'fields': ['id', 'x_studio_product_code']})
                
                # Create a mapping of product_id to x_studio_product_code
                product_code_map_v18 = {p['id']: p['x_studio_product_code'] for p in products_v18}
                
                # Organize by bom_id for easy lookup
                for line in all_bom_lines_v18:
                    bom_id = line['bom_id'][0] if line['bom_id'] else None
                    if bom_id not in bom_lines_v18_dict:
                        bom_lines_v18_dict[bom_id] = []
                    # Add product_code to the line data
                    product_id = line['product_id'][0] if line['product_id'] else None
                    line['product_code'] = product_code_map_v18.get(product_id, 'N/A')
                    bom_lines_v18_dict[bom_id].append(line)
        
        # Collect mismatches and matches
        mismatches = []
        matches = []
        
        for bom_v14 in boms_v14:
            product_tmpl_name = bom_v14['product_tmpl_id'][1] if bom_v14['product_tmpl_id'] else 'N/A'
            product_variant_name = bom_v14['product_id'][1] if bom_v14['product_id'] else 'N/A (Template BOM)'
            bom_reference = bom_v14['code'] if bom_v14['code'] else 'No reference'
            bom_display_name = bom_v14['display_name'] if bom_v14['display_name'] else 'No display name'
            bom_id_v14 = str(bom_v14['id'])
            
            # Get BOM line components from V14 (from pre-fetched data)
            components_v14 = {}
            bom_lines = bom_lines_v14_dict.get(bom_v14['id'], [])
            
            for line in bom_lines:
                component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                component_code = line['product_code']
                component_qty = line['product_qty']
                # Use normalized code as key for comparison
                normalized_code = normalize_code(component_code)
                components_v14[normalized_code] = {
                    'name': component_name, 
                    'qty': component_qty,
                    'original_code': component_code
                }
            
            # Find corresponding BOM in V18 by matching V14 ID to V18 code (reference)
            bom_v18 = boms_v18_dict.get(bom_id_v14)
            
            if not bom_v18:
                mismatches.append({
                    'type': 'missing_v18',
                    'bom_v14': bom_v14,
                    'bom_id_v14': bom_id_v14,
                    'bom_reference': bom_reference,
                    'bom_display_name': bom_display_name,
                    'product_tmpl_name': product_tmpl_name,
                    'product_variant_name': product_variant_name,
                    'bom_lines': bom_lines
                })
                continue
            
            # Get BOM line components from V18 (from pre-fetched data)
            components_v18 = {}
            bom_lines_v18 = bom_lines_v18_dict.get(bom_v18['id'], [])
            
            for line in bom_lines_v18:
                component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                component_code = line['product_code']
                component_qty = line['product_qty']
                # Use normalized code as key for comparison
                normalized_code = normalize_code(component_code)
                components_v18[normalized_code] = {
                    'name': component_name, 
                    'qty': component_qty,
                    'original_code': component_code
                }
            
            # Compare components
            has_mismatch = False
            mismatch_details = []
            
            # Check for missing components in V18
            for normalized_code, component_data in components_v14.items():
                if normalized_code not in components_v18:
                    mismatch_details.append(f"    ❌ MISSING IN V18: {component_data['name']} (Code: {component_data['original_code']}, Qty in V14: {component_data['qty']})")
                    has_mismatch = True
                elif components_v18[normalized_code]['qty'] != component_data['qty']:
                    mismatch_details.append(f"    ⚠️  QUANTITY MISMATCH: {component_data['name']} (Code: {component_data['original_code']})")
                    mismatch_details.append(f"       V14 Qty: {component_data['qty']}")
                    mismatch_details.append(f"       V18 Qty: {components_v18[normalized_code]['qty']}")
                    has_mismatch = True
            
            # Check for extra components in V18
            for normalized_code, component_data in components_v18.items():
                if normalized_code not in components_v14:
                    mismatch_details.append(f"    ➕ EXTRA IN V18: {component_data['name']} (Code: {component_data['original_code']}, Qty: {component_data['qty']})")
                    has_mismatch = True
            
            if has_mismatch:
                mismatches.append({
                    'type': 'component_mismatch',
                    'bom_v14': bom_v14,
                    'bom_v18': bom_v18,
                    'bom_id_v14': bom_id_v14,
                    'bom_reference': bom_reference,
                    'bom_display_name': bom_display_name,
                    'product_tmpl_name': product_tmpl_name,
                    'product_variant_name': product_variant_name,
                    'bom_lines': bom_lines,
                    'bom_lines_v18': bom_lines_v18,
                    'mismatch_details': mismatch_details
                })
            else:
                matches.append({
                    'bom_v14': bom_v14,
                    'bom_v18': bom_v18,
                    'bom_id_v14': bom_id_v14,
                    'bom_reference': bom_reference,
                    'bom_display_name': bom_display_name,
                    'product_tmpl_name': product_tmpl_name,
                    'product_variant_name': product_variant_name,
                    'bom_lines': bom_lines,
                    'bom_lines_v18': bom_lines_v18
                })
        
        # Print mismatches first
        if mismatches:
            print("\n" + "=" * 80)
            print("MISMATCHES FOUND:")
            print("=" * 80)
            
            for mismatch in mismatches:
                print(f"\n{'=' * 80}")
                print(f"BOM ID (V14): {mismatch['bom_v14']['id']}")
                print(f"  Reference: {mismatch['bom_reference']}")
                print(f"  Display Name: {mismatch['bom_display_name']}")
                print(f"  Product Template: {mismatch['product_tmpl_name']}")
                print(f"  Product Variant: {mismatch['product_variant_name']}")
                print(f"  is_custom_made (V14): {mismatch['bom_v14']['is_custom_made']}")
                
                if mismatch['type'] == 'missing_v18':
                    print(f"\n  ❌ NO CORRESPONDING BOM FOUND IN V18 (searched by code: '{mismatch['bom_id_v14']}')!")
                else:
                    print(f"\n  ✅ Found corresponding BOM in V18 (ID: {mismatch['bom_v18']['id']}, Code: '{mismatch['bom_v18']['code']}')")
                    
                    if mismatch['bom_lines']:
                        print(f"\n  [V14] Components ({len(mismatch['bom_lines'])} items):")
                        for line in mismatch['bom_lines']:
                            component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                            component_code = line['product_code']
                            component_qty = line['product_qty']
                            print(f"    - {component_name} (Code: {component_code}, Qty: {component_qty})")
                    else:
                        print(f"\n  [V14] Components: None")
                    
                    if mismatch['bom_lines_v18']:
                        print(f"\n  [V18] Components ({len(mismatch['bom_lines_v18'])} items):")
                        for line in mismatch['bom_lines_v18']:
                            component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                            component_code = line['product_code']
                            component_qty = line['product_qty']
                            print(f"    - {component_name} (Code: {component_code}, Qty: {component_qty})")
                    else:
                        print(f"\n  [V18] Components: None")
                    
                    print(f"\n  🔍 COMPARISON RESULTS:")
                    for detail in mismatch['mismatch_details']:
                        print(detail)
        
        # Print matches
        if matches:
            print("\n" + "=" * 80)
            print("MATCHES FOUND:")
            print("=" * 80)
            
            for match in matches:
                print(f"\n{'=' * 80}")
                print(f"BOM ID (V14): {match['bom_v14']['id']}")
                print(f"  Reference: {match['bom_reference']}")
                print(f"  Display Name: {match['bom_display_name']}")
                print(f"  Product Template: {match['product_tmpl_name']}")
                print(f"  Product Variant: {match['product_variant_name']}")
                print(f"  is_custom_made (V14): {match['bom_v14']['is_custom_made']}")
                print(f"\n  ✅ Found corresponding BOM in V18 (ID: {match['bom_v18']['id']}, Code: '{match['bom_v18']['code']}')")
                
                if match['bom_lines']:
                    print(f"\n  [V14] Components ({len(match['bom_lines'])} items):")
                    for line in match['bom_lines']:
                        component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                        component_code = line['product_code']
                        component_qty = line['product_qty']
                        print(f"    - {component_name} (Code: {component_code}, Qty: {component_qty})")
                else:
                    print(f"\n  [V14] Components: None")
                
                if match['bom_lines_v18']:
                    print(f"\n  [V18] Components ({len(match['bom_lines_v18'])} items):")
                    for line in match['bom_lines_v18']:
                        component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                        component_code = line['product_code']
                        component_qty = line['product_qty']
                        print(f"    - {component_name} (Code: {component_code}, Qty: {component_qty})")
                else:
                    print(f"\n  [V18] Components: None")
                
                print(f"\n  ✅ All components match perfectly!")
    
    print("\n" + "=" * 80)
    print(f"BOM comparison complete!")
    print(f"Total BOMs checked: {len(bom_ids_v14) if bom_ids_v14 else 0}")
    print(f"BOMs with mismatches: {len(mismatches) if 'mismatches' in locals() else 0}")
    print(f"BOMs with perfect matches: {len(matches) if 'matches' in locals() else 0}")
    print("=" * 80)
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


Comparing BOMs between Odoo V14 and V18

[V14] Authenticating...
✅ V14 Authenticated! UID: 28

[V18] Authenticating...
✅ V18 Authenticated! UID: 2

Checking specific categories: [4, 8, 10, 57, 82]

Categories to check:
  - Category ID 4: 0. Presets / 1. Presets zonder PP
  - Category ID 8: 0. Presets / 2. Presets met PP
  - Category ID 10: 0. Presets / 3. Losse verkopen
  - Category ID 57: 0. Presets / 4. Losse PP
  - Category ID 82: 0. Presets / Fixed margin

[V14] Fetching product templates in these categories...
Found 28 product templates

[V14] Searching for BOMs with is_custom_made = False...

[V14] Found 249 BOMs with is_custom_made = False


MISMATCHES FOUND:

BOM ID (V14): 92
  Reference: Aluminium Spieraam met opspannen (25mm, Met tussenlat)
  Display Name: Aluminium Spieraam met opspannen (25mm, Met tussenlat): Spieraam met opspannen
  Product Template: Spieraam met opspannen
  Product Variant: Spieraam met opspannen (25mm, Met tussenlat)
  is_custom_made (V14): False

  ❌ NO

In [64]:
# Compare material_cost (V14) vs x_studio_computed_cost (V18) for product templates

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Get environment variables for V14
JUSTFRAMEIT_ODOO_V14_URL = os.getenv('JUSTFRAMEIT_ODOO_V14_URL')
JUSTFRAMEIT_ODOO_V14_DB = os.getenv('JUSTFRAMEIT_ODOO_V14_DB')
JUSTFRAMEIT_ODOO_V14_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_V14_USERNAME')
JUSTFRAMEIT_ODOO_V14_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_V14_API_KEY')

# Get environment variables for V18
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

print("=" * 80)
print("Comparing material_cost (V14) vs x_studio_computed_cost (V18) for Product Templates")
print("=" * 80)

try:
    # Authenticate to V14
    print("\n[V14] Authenticating...")
    common_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/common", allow_none=True)
    uid_v14 = common_v14.authenticate(JUSTFRAMEIT_ODOO_V14_DB, JUSTFRAMEIT_ODOO_V14_USERNAME, JUSTFRAMEIT_ODOO_V14_API_KEY, {})
    print(f"✅ V14 Authenticated! UID: {uid_v14}")
    
    # Authenticate to V18
    print("\n[V18] Authenticating...")
    common_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
    uid_v18 = common_v18.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
    print(f"✅ V18 Authenticated! UID: {uid_v18}\n")
    
    # Create models proxies
    models_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/object", allow_none=True)
    models_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)
    
    # Define category IDs to EXCLUDE
    excluded_category_ids = [1, 4, 8, 10, 57, 82]
    
    print(f"Excluding categories: {excluded_category_ids}\n")
    
    # Get category details from V14
    categories = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.category', 'read',
        [excluded_category_ids],
        {'fields': ['display_name']})
    
    print("Categories to exclude:")
    for cat in categories:
        print(f"  - Category ID {cat['id']}: {cat['display_name']}")
    print()
    
    # Get all product templates from V14 excluding the specified categories
    print("[V14] Fetching product templates (excluding specified categories)...")
    template_ids_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.template', 'search',
        [[['categ_id', 'not in', excluded_category_ids]]])
    
    print(f"Found {len(template_ids_v14)} product templates in V14\n")
    
    # Get product template details from V14
    print("[V14] Reading product template details (id, product_code, material_cost, categ_id)...")
    templates_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.template', 'read',
        [template_ids_v14],
        {'fields': ['id', 'default_code', 'product_code', 'name', 'material_cost', 'categ_id']})
    
    print(f"Retrieved details for {len(templates_v14)} product templates\n")
    
    # Get all product codes from V14 (excluding None values)
    product_codes_v14 = [t['product_code'] for t in templates_v14 if t.get('product_code')]
    
    print(f"Found {len(product_codes_v14)} product templates with product_code in V14\n")
    
    # Fetch ALL matching product templates from V18 in ONE API call using product codes
    print("[V18] Fetching all matching product templates by x_studio_product_code in one call...")
    template_ids_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
        'product.template', 'search',
        [[['x_studio_product_code', 'in', product_codes_v14]]])
    
    print(f"Found {len(template_ids_v18)} matching product templates in V18\n")
    
    # Get all V18 product template details in ONE API call
    print("[V18] Reading all product template details in one call...")
    templates_v18_list = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
        'product.template', 'read',
        [template_ids_v18],
        {'fields': ['id', 'default_code', 'x_studio_product_code', 'name', 'x_studio_computed_cost']})
    
    # Create a dictionary for quick lookup by x_studio_product_code
    templates_v18_dict = {t['x_studio_product_code']: t for t in templates_v18_list if t.get('x_studio_product_code')}
    
    print(f"Retrieved details for {len(templates_v18_list)} product templates from V18\n")
    
    print("Comparing costs between V14 and V18:")
    print("-" * 80)
    
    mismatches_found = 0
    matches_found = 0
    templates_checked = 0
    templates_not_found_in_v18 = 0
    templates_without_product_code = 0
    
    for template_v14 in templates_v14:
        template_id = template_v14['id']
        template_code = template_v14['default_code']
        product_code_v14 = template_v14.get('product_code')
        template_name = template_v14['name']
        material_cost_v14 = template_v14['material_cost'] if template_v14['material_cost'] else 0.0
        category_name = template_v14['categ_id'][1] if template_v14['categ_id'] else 'N/A'
        
        # Skip if no product_code in V14
        if not product_code_v14:
            templates_without_product_code += 1
            continue
        
        templates_checked += 1
        
        # Look up product template in V18 dictionary by product_code
        template_v18 = templates_v18_dict.get(product_code_v14)
        
        if not template_v18:
            print(f"\n❌ Product Template NOT FOUND in V18:")
            print(f"   ID (V14): {template_id}")
            print(f"   Code: {template_code}")
            print(f"   product_code (V14): {product_code_v14}")
            print(f"   Name: {template_name}")
            print(f"   Category: {category_name}")
            print(f"   material_cost (V14): €{material_cost_v14:.2f}")
            templates_not_found_in_v18 += 1
            continue
        
        computed_cost_v18 = template_v18['x_studio_computed_cost'] if template_v18['x_studio_computed_cost'] else 0.0
        
        # Compare costs (with small tolerance for floating point comparison)
        tolerance = 0.01
        if abs(material_cost_v14 - computed_cost_v18) > tolerance:
            print(f"\n⚠️  COST MISMATCH:")
            print(f"   ID (V14): {template_id}")
            print(f"   ID (V18): {template_v18['id']}")
            print(f"   Code: {template_code}")
            print(f"   product_code (V14): {product_code_v14}")
            print(f"   x_studio_product_code (V18): {template_v18['x_studio_product_code']}")
            print(f"   Name (V14): {template_name}")
            print(f"   Name (V18): {template_v18['name']}")
            print(f"   Category: {category_name}")
            print(f"   material_cost (V14): €{material_cost_v14:.2f}")
            print(f"   x_studio_computed_cost (V18): €{computed_cost_v18:.2f}")
            print(f"   Difference: €{abs(material_cost_v14 - computed_cost_v18):.2f}")
            mismatches_found += 1
        else:
            matches_found += 1
    
    print("\n" + "=" * 80)
    print(f"Cost comparison complete!")
    print(f"Total product templates in V14 (excluding categories): {len(templates_v14)}")
    print(f"Product templates without product_code (skipped): {templates_without_product_code}")
    print(f"Product templates checked: {templates_checked}")
    print(f"Product templates not found in V18: {templates_not_found_in_v18}")
    print(f"Product templates with cost matches: {matches_found}")
    print(f"Product templates with cost mismatches: {mismatches_found}")
    print("=" * 80)
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


Comparing material_cost (V14) vs x_studio_computed_cost (V18) for Product Templates

[V14] Authenticating...
✅ V14 Authenticated! UID: 28

[V18] Authenticating...
✅ V18 Authenticated! UID: 2

Excluding categories: [1, 4, 8, 10, 57, 82]

Categories to exclude:
  - Category ID 1: 0. Presets
  - Category ID 4: 0. Presets / 1. Presets zonder PP
  - Category ID 8: 0. Presets / 2. Presets met PP
  - Category ID 10: 0. Presets / 3. Losse verkopen
  - Category ID 57: 0. Presets / 4. Losse PP
  - Category ID 82: 0. Presets / Fixed margin

[V14] Fetching product templates (excluding specified categories)...
Found 3318 product templates in V14

[V14] Reading product template details (id, product_code, material_cost, categ_id)...
Retrieved details for 3318 product templates

Found 3292 product templates with product_code in V14

[V18] Fetching all matching product templates by x_studio_product_code in one call...
Found 3234 matching product templates in V18

[V18] Reading all product template deta

In [ ]:
# Compare BOMs between Odoo V14 and V18 to find mismatches

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Get environment variables for V14
JUSTFRAMEIT_ODOO_V14_URL = os.getenv('JUSTFRAMEIT_ODOO_V14_URL')
JUSTFRAMEIT_ODOO_V14_DB = os.getenv('JUSTFRAMEIT_ODOO_V14_DB')
JUSTFRAMEIT_ODOO_V14_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_V14_USERNAME')
JUSTFRAMEIT_ODOO_V14_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_V14_API_KEY')

# Get environment variables for V18
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

print("=" * 80)
print("Comparing BOMs between Odoo V14 and V18")
print("=" * 80)

try:
    # Authenticate to V14
    print("\n[V14] Authenticating...")
    common_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/common", allow_none=True)
    uid_v14 = common_v14.authenticate(JUSTFRAMEIT_ODOO_V14_DB, JUSTFRAMEIT_ODOO_V14_USERNAME, JUSTFRAMEIT_ODOO_V14_API_KEY, {})
    print(f"✅ V14 Authenticated! UID: {uid_v14}")
    
    # Authenticate to V18
    print("\n[V18] Authenticating...")
    common_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
    uid_v18 = common_v18.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
    print(f"✅ V18 Authenticated! UID: {uid_v18}\n")
    
    # Create models proxies
    models_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/object", allow_none=True)
    models_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)
    
    # Define specific category IDs to check
    target_category_ids = [4, 8, 10, 57, 82]
    #target_category_ids = [10] #for testing
    
    print(f"Checking specific categories: {target_category_ids}\n")
    
    # Get category details from V14 (single call)
    categories = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.category', 'read',
        [target_category_ids],
        {'fields': ['display_name']})
    
    print("Categories to check:")
    for cat in categories:
        print(f"  - Category ID {cat['id']}: {cat['display_name']}")
    print()
    
    # Get all product templates in these categories from V14 (single call)
    print("[V14] Fetching product templates in these categories...")
    product_template_ids = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.template', 'search',
        [[['categ_id', 'in', target_category_ids]]])
    
    print(f"Found {len(product_template_ids)} product templates\n")
    
    # Search for BOMs with is_custom_made = False in V14 (single call) - including inactive BOMs
    print("[V14] Searching for BOMs with is_custom_made = False (including inactive BOMs)...")
    print("=" * 80)
    
    bom_ids_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'mrp.bom', 'search',
        [[['product_tmpl_id', 'in', product_template_ids], ['is_custom_made', '=', False]]],
        {'context': {'active_test': False}})
    
    print(f"\n[V14] Found {len(bom_ids_v14)} BOMs with is_custom_made = False (including inactive)\n")
    
    if not bom_ids_v14:
        print("No BOMs found with is_custom_made = False in V14")
    else:
        # Get ALL BOM details from V14 in a single call (including active field)
        boms_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
            'mrp.bom', 'read',
            [bom_ids_v14],
            {'fields': ['id', 'code', 'display_name', 'product_tmpl_id', 'product_id', 'is_custom_made', 'bom_line_ids', 'active']})
        
        # Get ALL BOM lines from V14 in a single call
        all_bom_line_ids_v14 = []
        for bom in boms_v14:
            if bom['bom_line_ids']:
                all_bom_line_ids_v14.extend(bom['bom_line_ids'])
        
        bom_lines_v14_dict = {}
        if all_bom_line_ids_v14:
            all_bom_lines_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'mrp.bom.line', 'read',
                [all_bom_line_ids_v14],
                {'fields': ['id', 'product_id', 'product_qty', 'bom_id']})
            
            # Get all product IDs from BOM lines to fetch product codes
            product_ids_v14 = [line['product_id'][0] for line in all_bom_lines_v14 if line['product_id']]
            
            # Fetch product codes for all products in V14
            products_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'product.product', 'read',
                [product_ids_v14],
                {'fields': ['id', 'product_code']})
            
            # Create a mapping of product_id to product_code
            product_code_map_v14 = {p['id']: p['product_code'] for p in products_v14}
            
            # Organize by bom_id for easy lookup
            for line in all_bom_lines_v14:
                bom_id = line['bom_id'][0] if line['bom_id'] else None
                if bom_id not in bom_lines_v14_dict:
                    bom_lines_v14_dict[bom_id] = []
                # Add product_code to the line data
                product_id = line['product_id'][0] if line['product_id'] else None
                line['product_code'] = product_code_map_v14.get(product_id, 'N/A')
                bom_lines_v14_dict[bom_id].append(line)
        
        # Collect all BOM codes to search in V18
        bom_codes_v14 = [bom['code'] for bom in boms_v14 if bom['code']]
        
        # Search for ALL corresponding BOMs in V18 in a single call (including inactive)
        bom_ids_v18 = []
        if bom_codes_v14:
            bom_ids_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'mrp.bom', 'search',
                [[['code', 'in', bom_codes_v14]]],
                {'context': {'active_test': False}})
        
        # Get ALL BOM details from V18 in a single call (including active field)
        boms_v18_dict = {}
        if bom_ids_v18:
            boms_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'mrp.bom', 'read',
                [bom_ids_v18],
                {'fields': ['id', 'code', 'display_name', 'product_tmpl_id', 'product_id', 'bom_line_ids', 'active']})
            
            # Organize by code for easy lookup (handle duplicates by storing as list)
            for bom in boms_v18:
                if bom['code']:
                    if bom['code'] not in boms_v18_dict:
                        boms_v18_dict[bom['code']] = []
                    boms_v18_dict[bom['code']].append(bom)
            
            # Get ALL BOM lines from V18 in a single call
            all_bom_line_ids_v18 = []
            for bom in boms_v18:
                if bom['bom_line_ids']:
                    all_bom_line_ids_v18.extend(bom['bom_line_ids'])
            
            bom_lines_v18_dict = {}
            if all_bom_line_ids_v18:
                all_bom_lines_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'mrp.bom.line', 'read',
                    [all_bom_line_ids_v18],
                    {'fields': ['id', 'product_id', 'product_qty', 'bom_id']})
                
                # Get all product IDs from BOM lines to fetch product codes
                product_ids_v18 = [line['product_id'][0] for line in all_bom_lines_v18 if line['product_id']]
                
                # Fetch product codes for all products in V18
                products_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'product.product', 'read',
                    [product_ids_v18],
                    {'fields': ['id', 'x_studio_product_code']})
                
                # Create a mapping of product_id to x_studio_product_code
                product_code_map_v18 = {p['id']: p['x_studio_product_code'] for p in products_v18}
                
                # Organize by bom_id for easy lookup
                for line in all_bom_lines_v18:
                    bom_id = line['bom_id'][0] if line['bom_id'] else None
                    if bom_id not in bom_lines_v18_dict:
                        bom_lines_v18_dict[bom_id] = []
                    # Add product_code to the line data
                    product_id = line['product_id'][0] if line['product_id'] else None
                    line['product_code'] = product_code_map_v18.get(product_id, 'N/A')
                    bom_lines_v18_dict[bom_id].append(line)
        
        # Fetch product codes for all product_ids in V14 BOMs for fallback matching
        all_product_ids_v14 = [bom['product_id'][0] for bom in boms_v14 if bom['product_id']]
        product_codes_v14_map = {}
        product_variants_v14_map = {}
        if all_product_ids_v14:
            products_for_boms_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'product.product', 'read',
                [all_product_ids_v14],
                {'fields': ['id', 'product_code', 'display_name']})
            product_codes_v14_map = {p['id']: p['product_code'] for p in products_for_boms_v14}
            product_variants_v14_map = {p['id']: p['display_name'] for p in products_for_boms_v14}
        
        # Fetch product variants for all product_ids in V18 BOMs for fallback matching
        all_product_ids_v18 = []
        for bom_list in boms_v18_dict.values():
            for bom in bom_list:
                if bom['product_id']:
                    all_product_ids_v18.append(bom['product_id'][0])
        
        product_variants_v18_map = {}
        if all_product_ids_v18:
            products_for_boms_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'product.product', 'read',
                [all_product_ids_v18],
                {'fields': ['id', 'x_studio_product_code', 'display_name']})
            product_variants_v18_map = {p['id']: {'code': p['x_studio_product_code'], 'name': p['display_name']} for p in products_for_boms_v18}
        
        print("Comparing BOMs between V14 and V18:")
        print("-" * 80)
        
        mismatches_found = 0
        
        for bom_v14 in boms_v14:
            product_tmpl_name = bom_v14['product_tmpl_id'][1] if bom_v14['product_tmpl_id'] else 'N/A'
            product_variant_name = bom_v14['product_id'][1] if bom_v14['product_id'] else 'N/A (Template BOM)'
            bom_reference = bom_v14['code'] if bom_v14['code'] else 'No reference'
            bom_display_name = bom_v14['display_name'] if bom_v14['display_name'] else 'No display name'
            bom_active_v14 = bom_v14.get('active', True)
            
            print(f"\n{'=' * 80}")
            print(f"BOM ID (V14): {bom_v14['id']}")
            print(f"  Reference: {bom_reference}")
            print(f"  Display Name: {bom_display_name}")
            print(f"  Product Template: {product_tmpl_name}")
            print(f"  Product Variant: {product_variant_name}")
            print(f"  is_custom_made (V14): {bom_v14['is_custom_made']}")
            print(f"  active (V14): {bom_active_v14}")
            
            # Get BOM line components from V14 (from pre-fetched data)
            components_v14 = {}
            bom_lines = bom_lines_v14_dict.get(bom_v14['id'], [])
            
            if bom_lines:
                print(f"\n  [V14] Components ({len(bom_lines)} items):")
                for line in bom_lines:
                    component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                    component_code = line['product_code']
                    component_qty = line['product_qty']
                    components_v14[component_code] = {'name': component_name, 'qty': component_qty}
                    print(f"    - {component_name} (Code: {component_code}, Qty: {component_qty})")
            else:
                print(f"\n  [V14] Components: None")
            
            # Find corresponding BOM in V18
            print(f"\n  [V18] Searching for corresponding BOM...")
            
            bom_v18 = None
            matching_method = None
            
            # Try matching by BOM reference (code) first
            if bom_reference and bom_reference != 'No reference':
                print(f"     Trying to match by BOM reference (code): {bom_reference}")
                
                matching_boms = boms_v18_dict.get(bom_reference, [])
                
                if len(matching_boms) == 1:
                    # Unique match by code
                    bom_v18 = matching_boms[0]
                    matching_method = "by BOM reference (code)"
                    print(f"  ✅ Found unique BOM in V18 by reference")
                elif len(matching_boms) > 1:
                    # Multiple BOMs with same code - need to use product_id fallback
                    print(f"  ⚠️  Found {len(matching_boms)} BOMs with same reference in V18, using product_id fallback")
                    
                    # Get product_code from V14 product_id
                    product_id_v14 = bom_v14['product_id'][0] if bom_v14['product_id'] else None
                    if product_id_v14:
                        product_code_v14 = product_codes_v14_map.get(product_id_v14)
                        product_variant_v14 = product_variants_v14_map.get(product_id_v14)
                        if product_code_v14:
                            print(f"     V14 product_code: {product_code_v14}")
                            print(f"     V14 product_variant: {product_variant_v14}")
                            
                            # Try to match by product_code in V18
                            for candidate_bom in matching_boms:
                                if candidate_bom['product_id']:
                                    product_id_v18 = candidate_bom['product_id'][0]
                                    product_info_v18 = product_variants_v18_map.get(product_id_v18, {})
                                    product_code_v18 = product_info_v18.get('code')
                                    product_variant_v18 = product_info_v18.get('name')
                                    
                                    print(f"       Checking candidate: product_code={product_code_v18}, variant={product_variant_v18}")
                                    
                                    if product_code_v18 == product_code_v14:
                                        bom_v18 = candidate_bom
                                        matching_method = "by BOM reference + product_code fallback"
                                        print(f"  ✅ Matched by product_code: {product_code_v18}")
                                        break
                            
                            # If still not matched, try comparing product variants (display_name)
                            if not bom_v18 and product_variant_v14:
                                print(f"     Trying to match by product variant display_name")
                                for candidate_bom in matching_boms:
                                    if candidate_bom['product_id']:
                                        product_id_v18 = candidate_bom['product_id'][0]
                                        product_info_v18 = product_variants_v18_map.get(product_id_v18, {})
                                        product_variant_v18 = product_info_v18.get('name')
                                        
                                        if product_variant_v18 == product_variant_v14:
                                            bom_v18 = candidate_bom
                                            matching_method = "by BOM reference + product variant display_name fallback"
                                            print(f"  ✅ Matched by product variant: {product_variant_v18}")
                                            break
                            
                            if not bom_v18:
                                print(f"  ❌ Could not match any of the {len(matching_boms)} BOMs by product_code or product variant")
                        else:
                            print(f"  ❌ Could not get product_code from V14 product_id")
                    else:
                        print(f"  ❌ No product_id in V14 BOM for fallback matching")
            
            # Fallback: try matching by product_code if no reference or reference didn't match
            if not bom_v18:
                print(f"     Trying fallback: matching by product_code")
                
                product_id_v14 = bom_v14['product_id'][0] if bom_v14['product_id'] else None
                if product_id_v14:
                    product_code_v14 = product_codes_v14_map.get(product_id_v14)
                    product_variant_v14 = product_variants_v14_map.get(product_id_v14)
                    if product_code_v14:
                        print(f"     V14 product_code: {product_code_v14}")
                        print(f"     V14 product_variant: {product_variant_v14}")
                        
                        # Search through all V18 BOMs for matching product_code
                        for code, bom_list in boms_v18_dict.items():
                            for candidate_bom in bom_list:
                                if candidate_bom['product_id']:
                                    product_id_v18 = candidate_bom['product_id'][0]
                                    product_info_v18 = product_variants_v18_map.get(product_id_v18, {})
                                    product_code_v18 = product_info_v18.get('code')
                                    if product_code_v18 == product_code_v14:
                                        bom_v18 = candidate_bom
                                        matching_method = "by product_code fallback"
                                        print(f"  ✅ Found BOM in V18 by product_code: {product_code_v18}")
                                        break
                            if bom_v18:
                                break
                        
                        # If still not matched, try comparing product variants (display_name)
                        if not bom_v18 and product_variant_v14:
                            print(f"     Trying to match by product variant display_name")
                            for code, bom_list in boms_v18_dict.items():
                                for candidate_bom in bom_list:
                                    if candidate_bom['product_id']:
                                        product_id_v18 = candidate_bom['product_id'][0]
                                        product_info_v18 = product_variants_v18_map.get(product_id_v18, {})
                                        product_variant_v18 = product_info_v18.get('name')
                                        
                                        if product_variant_v18 == product_variant_v14:
                                            bom_v18 = candidate_bom
                                            matching_method = "by product variant display_name fallback"
                                            print(f"  ✅ Found BOM in V18 by product variant: {product_variant_v18}")
                                            break
                                if bom_v18:
                                    break
                    else:
                        print(f"  ❌ Could not get product_code from V14 product_id")
                else:
                    print(f"  ❌ No product_id in V14 BOM for fallback matching")
            
            if not bom_v18:
                print(f"  ❌ NO CORRESPONDING BOM FOUND IN V18!")
                mismatches_found += 1
                continue
            
            bom_active_v18 = bom_v18.get('active', True)
            print(f"  ✅ Found corresponding BOM in V18 (ID: {bom_v18['id']}, Code: '{bom_v18['code']}', active: {bom_active_v18}, matched {matching_method})")
            
            # Get BOM line components from V18 (from pre-fetched data)
            components_v18 = {}
            bom_lines_v18 = bom_lines_v18_dict.get(bom_v18['id'], [])
            
            if bom_lines_v18:
                print(f"\n  [V18] Components ({len(bom_lines_v18)} items):")
                for line in bom_lines_v18:
                    component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                    component_code = line['product_code']
                    component_qty = line['product_qty']
                    components_v18[component_code] = {'name': component_name, 'qty': component_qty}
                    print(f"    - {component_name} (Code: {component_code}, Qty: {component_qty})")
            else:
                print(f"\n  [V18] Components: None")
            
            # Compare components
            print(f"\n  🔍 COMPARISON RESULTS:")
            has_mismatch = False
            
            # Check for missing components in V18
            for component_code, component_data in components_v14.items():
                if component_code not in components_v18:
                    print(f"    ❌ MISSING IN V18: {component_data['name']} (Code: {component_code}, Qty in V14: {component_data['qty']})")
                    has_mismatch = True
                elif components_v18[component_code]['qty'] != component_data['qty']:
                    print(f"    ⚠️  QUANTITY MISMATCH: {component_data['name']} (Code: {component_code})")
                    print(f"       V14 Qty: {component_data['qty']}")
                    print(f"       V18 Qty: {components_v18[component_code]['qty']}")
                    has_mismatch = True
            
            # Check for extra components in V18
            for component_code, component_data in components_v18.items():
                if component_code not in components_v14:
                    print(f"    ➕ EXTRA IN V18: {component_data['name']} (Code: {component_code}, Qty: {component_data['qty']})")
                    has_mismatch = True
            
            if not has_mismatch:
                print(f"    ✅ All components match!")
            else:
                mismatches_found += 1
    
    print("\n" + "=" * 80)
    print(f"BOM comparison complete!")
    print(f"Total BOMs checked: {len(bom_ids_v14) if bom_ids_v14 else 0}")
    print(f"BOMs with mismatches: {mismatches_found}")
    print("=" * 80)
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


Comparing BOMs between Odoo V14 and V18

[V14] Authenticating...
✅ V14 Authenticated! UID: 28

[V18] Authenticating...
✅ V18 Authenticated! UID: 2

Checking specific categories: [4, 8, 10, 57, 82]

Categories to check:
  - Category ID 4: 0. Presets / 1. Presets zonder PP
  - Category ID 8: 0. Presets / 2. Presets met PP
  - Category ID 10: 0. Presets / 3. Losse verkopen
  - Category ID 57: 0. Presets / 4. Losse PP
  - Category ID 82: 0. Presets / Fixed margin

[V14] Fetching product templates in these categories...
Found 28 product templates

[V14] Searching for BOMs with is_custom_made = False (including inactive BOMs)...

[V14] Found 250 BOMs with is_custom_made = False (including inactive)

Comparing BOMs between V14 and V18:
--------------------------------------------------------------------------------

BOM ID (V14): 49
  Reference: Los glas (Blinkend glas)
  Display Name: Los glas (Blinkend glas): Los glas
  Product Template: Los glas
  Product Variant: [P10P] Los glas (Blinkend

In [16]:
# Compare BOMs between Odoo V14 and V18 to find mismatches

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Get environment variables for V14
JUSTFRAMEIT_ODOO_V14_URL = os.getenv('JUSTFRAMEIT_ODOO_V14_URL')
JUSTFRAMEIT_ODOO_V14_DB = os.getenv('JUSTFRAMEIT_ODOO_V14_DB')
JUSTFRAMEIT_ODOO_V14_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_V14_USERNAME')
JUSTFRAMEIT_ODOO_V14_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_V14_API_KEY')

# Get environment variables for V18
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

print("=" * 80)
print("Comparing BOMs between Odoo V14 and V18")
print("=" * 80)

try:
    # Authenticate to V14
    print("\n[V14] Authenticating...")
    common_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/common", allow_none=True)
    uid_v14 = common_v14.authenticate(JUSTFRAMEIT_ODOO_V14_DB, JUSTFRAMEIT_ODOO_V14_USERNAME, JUSTFRAMEIT_ODOO_V14_API_KEY, {})
    print(f"✅ V14 Authenticated! UID: {uid_v14}")
    
    # Authenticate to V18
    print("\n[V18] Authenticating...")
    common_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
    uid_v18 = common_v18.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
    print(f"✅ V18 Authenticated! UID: {uid_v18}\n")
    
    # Create models proxies
    models_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/object", allow_none=True)
    models_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)
    
    # Define specific category IDs to check
    target_category_ids = [4, 8, 10, 57, 82]
    #target_category_ids = [10] #for testing
    
    print(f"Checking specific categories: {target_category_ids}\n")
    
    # Get category details from V14 (single call)
    categories = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.category', 'read',
        [target_category_ids],
        {'fields': ['display_name']})
    
    print("Categories to check:")
    for cat in categories:
        print(f"  - Category ID {cat['id']}: {cat['display_name']}")
    print()
    
    # Get all product templates in these categories from V14 (single call)
    print("[V14] Fetching product templates in these categories...")
    product_template_ids = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.template', 'search',
        [[['categ_id', 'in', target_category_ids]]])
    
    print(f"Found {len(product_template_ids)} product templates\n")
    
    # Search for BOMs with is_custom_made = False in V14 (including inactive BOMs)
    print("[V14] Searching for BOMs with is_custom_made = False (including inactive)...")
    print("=" * 80)
    
    bom_ids_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'mrp.bom', 'search',
        [[['product_tmpl_id', 'in', product_template_ids], ['is_custom_made', '=', False]]],
        {'context': {'active_test': False}})
    
    print(f"\n[V14] Found {len(bom_ids_v14)} BOMs with is_custom_made = False (including inactive)\n")
    
    if not bom_ids_v14:
        print("No BOMs found with is_custom_made = False in V14")
    else:
        # Get ALL BOM details from V14 in a single call
        boms_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
            'mrp.bom', 'read',
            [bom_ids_v14],
            {'fields': ['id', 'code', 'display_name', 'product_tmpl_id', 'product_id', 'is_custom_made', 'bom_line_ids', 'active']})
        
        # Get ALL BOM lines from V14 in a single call
        all_bom_line_ids_v14 = []
        for bom in boms_v14:
            if bom['bom_line_ids']:
                all_bom_line_ids_v14.extend(bom['bom_line_ids'])
        
        bom_lines_v14_dict = {}
        if all_bom_line_ids_v14:
            all_bom_lines_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'mrp.bom.line', 'read',
                [all_bom_line_ids_v14],
                {'fields': ['id', 'product_id', 'product_qty', 'bom_id']})
            
            # Get all product IDs from BOM lines to fetch product codes
            product_ids_v14 = [line['product_id'][0] for line in all_bom_lines_v14 if line['product_id']]
            
            # Fetch product codes for all products in V14
            products_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'product.product', 'read',
                [product_ids_v14],
                {'fields': ['id', 'product_code']})
            
            # Create a mapping of product_id to product_code
            product_code_map_v14 = {p['id']: p['product_code'] for p in products_v14}
            
            # Organize by bom_id for easy lookup
            for line in all_bom_lines_v14:
                bom_id = line['bom_id'][0] if line['bom_id'] else None
                if bom_id not in bom_lines_v14_dict:
                    bom_lines_v14_dict[bom_id] = []
                # Add product_code to the line data
                product_id = line['product_id'][0] if line['product_id'] else None
                line['product_code'] = product_code_map_v14.get(product_id, 'N/A')
                bom_lines_v14_dict[bom_id].append(line)
        
        # Collect all BOM codes and product variant display names to search in V18
        bom_codes_v14 = [bom['code'] for bom in boms_v14 if bom['code']]
        
        # Get product variant IDs from V14 BOMs
        product_variant_ids_v14 = [bom['product_id'][0] for bom in boms_v14 if bom['product_id']]
        
        # Fetch display_name for all product variants in V14
        product_variants_v14 = []
        if product_variant_ids_v14:
            product_variants_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'product.product', 'read',
                [product_variant_ids_v14],
                {'fields': ['id', 'display_name']})
        
        # Create a mapping of product_id to display_name
        product_display_name_map_v14 = {p['id']: p['display_name'] for p in product_variants_v14}
        
        # Get all product variant display names
        product_variant_display_names_v14 = [product_display_name_map_v14.get(bom['product_id'][0]) for bom in boms_v14 if bom['product_id']]
        
        # Search for ALL corresponding BOMs in V18 by code
        bom_ids_v18_by_code = []
        if bom_codes_v14:
            bom_ids_v18_by_code = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'mrp.bom', 'search',
                [[['code', 'in', bom_codes_v14]]])
        
        # Search for ALL corresponding BOMs in V18 by product variant display name (in code field)
        bom_ids_v18_by_variant_in_code = []
        if product_variant_display_names_v14:
            bom_ids_v18_by_variant_in_code = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'mrp.bom', 'search',
                [[['code', 'in', product_variant_display_names_v14]]])
        
        # Search for ALL corresponding BOMs in V18 by product variant display name (in product_id field)
        bom_ids_v18_by_variant_in_product = []
        if product_variant_display_names_v14:
            # First get product IDs that match the variant display names
            product_ids_v18_matching = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'product.product', 'search',
                [[['display_name', 'in', product_variant_display_names_v14]]])
            
            if product_ids_v18_matching:
                bom_ids_v18_by_variant_in_product = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'mrp.bom', 'search',
                    [[['product_id', 'in', product_ids_v18_matching]]])
        
        # Combine all V18 BOM IDs
        all_bom_ids_v18 = list(set(bom_ids_v18_by_code + bom_ids_v18_by_variant_in_code + bom_ids_v18_by_variant_in_product))
        
        # Get ALL BOM details from V18 in a single call
        boms_v18_by_code = {}
        boms_v18_by_variant_display_name = {}
        if all_bom_ids_v18:
            boms_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'mrp.bom', 'read',
                [all_bom_ids_v18],
                {'fields': ['id', 'code', 'display_name', 'product_tmpl_id', 'product_id', 'bom_line_ids']})
            
            # Get product variant IDs from V18 BOMs to fetch display_name
            product_variant_ids_v18 = [bom['product_id'][0] for bom in boms_v18 if bom['product_id']]
            
            # Fetch display_name for all product variants in V18
            product_variants_v18 = []
            if product_variant_ids_v18:
                product_variants_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'product.product', 'read',
                    [product_variant_ids_v18],
                    {'fields': ['id', 'display_name']})
            
            # Create a mapping of product_id to display_name
            product_display_name_map_v18 = {p['id']: p['display_name'] for p in product_variants_v18}
            
            # Organize by code and by product variant display name for easy lookup
            for bom in boms_v18:
                if bom['code']:
                    if bom['code'] not in boms_v18_by_code:
                        boms_v18_by_code[bom['code']] = []
                    boms_v18_by_code[bom['code']].append(bom)
                
                if bom['product_id']:
                    variant_display_name = product_display_name_map_v18.get(bom['product_id'][0])
                    if variant_display_name:
                        if variant_display_name not in boms_v18_by_variant_display_name:
                            boms_v18_by_variant_display_name[variant_display_name] = []
                        boms_v18_by_variant_display_name[variant_display_name].append(bom)
            
            # Get ALL BOM lines from V18 in a single call
            all_bom_line_ids_v18 = []
            for bom in boms_v18:
                if bom['bom_line_ids']:
                    all_bom_line_ids_v18.extend(bom['bom_line_ids'])
            
            bom_lines_v18_dict = {}
            if all_bom_line_ids_v18:
                all_bom_lines_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'mrp.bom.line', 'read',
                    [all_bom_line_ids_v18],
                    {'fields': ['id', 'product_id', 'product_qty', 'bom_id']})
                
                # Get all product IDs from BOM lines to fetch product codes
                product_ids_v18 = [line['product_id'][0] for line in all_bom_lines_v18 if line['product_id']]
                
                # Fetch product codes for all products in V18
                products_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'product.product', 'read',
                    [product_ids_v18],
                    {'fields': ['id', 'x_studio_product_code']})
                
                # Create a mapping of product_id to x_studio_product_code
                product_code_map_v18 = {p['id']: p['x_studio_product_code'] for p in products_v18}
                
                # Organize by bom_id for easy lookup
                for line in all_bom_lines_v18:
                    bom_id = line['bom_id'][0] if line['bom_id'] else None
                    if bom_id not in bom_lines_v18_dict:
                        bom_lines_v18_dict[bom_id] = []
                    # Add product_code to the line data
                    product_id = line['product_id'][0] if line['product_id'] else None
                    line['product_code'] = product_code_map_v18.get(product_id, 'N/A')
                    bom_lines_v18_dict[bom_id].append(line)
        
        print("Comparing BOMs between V14 and V18:")
        print("-" * 80)
        
        mismatches_found = 0
        
        for bom_v14 in boms_v14:
            product_tmpl_name = bom_v14['product_tmpl_id'][1] if bom_v14['product_tmpl_id'] else 'N/A'
            product_variant_display_name = product_display_name_map_v14.get(bom_v14['product_id'][0]) if bom_v14['product_id'] else 'N/A (Template BOM)'
            bom_reference = bom_v14['code'] if bom_v14['code'] else 'No reference'
            bom_display_name = bom_v14['display_name'] if bom_v14['display_name'] else 'No display name'
            is_active = bom_v14.get('active', True)
            
            print(f"\n{'=' * 80}")
            print(f"BOM ID (V14): {bom_v14['id']}")
            print(f"  Reference: {bom_reference}")
            print(f"  Display Name: {bom_display_name}")
            print(f"  Product Template: {product_tmpl_name}")
            print(f"  Product Variant: {product_variant_display_name}")
            print(f"  is_custom_made (V14): {bom_v14['is_custom_made']}")
            print(f"  Active (V14): {is_active}")
            
            # Get BOM line components from V14 (from pre-fetched data)
            components_v14 = {}
            bom_lines = bom_lines_v14_dict.get(bom_v14['id'], [])
            
            if bom_lines:
                print(f"\n  [V14] Components ({len(bom_lines)} items):")
                for line in bom_lines:
                    component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                    component_code = line['product_code']
                    component_qty = line['product_qty']
                    components_v14[component_code] = {'name': component_name, 'qty': component_qty}
                    print(f"    - {component_name} (Code: {component_code}, Qty: {component_qty})")
            else:
                print(f"\n  [V14] Components: None")
            
            # Find corresponding BOM in V18 using multi-step matching strategy
            print(f"\n  [V18] Searching for corresponding BOM...")
            
            bom_v18 = None
            match_method = None
            
            # Step 1: Try matching by reference (code)
            if bom_reference and bom_reference != 'No reference':
                print(f"     Step 1: Trying to match by BOM reference (code): '{bom_reference}'")
                matching_boms = boms_v18_by_code.get(bom_reference, [])
                
                if len(matching_boms) == 1:
                    bom_v18 = matching_boms[0]
                    match_method = "reference (code)"
                    print(f"     ✅ Found unique match by reference")
                elif len(matching_boms) > 1:
                    print(f"     ⚠️  Found {len(matching_boms)} BOMs with same reference - trying next method")
                else:
                    print(f"     ❌ No match by reference")
            else:
                print(f"     Step 1: Skipped (no BOM reference in V14)")
            
            # Step 2: If no unique match, try matching product_id (variant display name) from V14 against code in V18
            if not bom_v18 and product_variant_display_name and product_variant_display_name != 'N/A (Template BOM)':
                print(f"     Step 2: Trying to match V14 product variant display name against V18 BOM code: '{product_variant_display_name}'")
                matching_boms = boms_v18_by_code.get(product_variant_display_name, [])
                
                if len(matching_boms) == 1:
                    bom_v18 = matching_boms[0]
                    match_method = "V14 variant display name vs V18 code"
                    print(f"     ✅ Found unique match")
                elif len(matching_boms) > 1:
                    print(f"     ⚠️  Found {len(matching_boms)} BOMs - trying next method")
                else:
                    print(f"     ❌ No match")
            
            # Step 3: If still no match, try matching product_id (variant display name) from V14 against product_id in V18
            if not bom_v18 and product_variant_display_name and product_variant_display_name != 'N/A (Template BOM)':
                print(f"     Step 3: Trying to match V14 product variant display name against V18 product variant display name: '{product_variant_display_name}'")
                matching_boms = boms_v18_by_variant_display_name.get(product_variant_display_name, [])
                
                if len(matching_boms) == 1:
                    bom_v18 = matching_boms[0]
                    match_method = "V14 variant display name vs V18 variant display name"
                    print(f"     ✅ Found unique match")
                elif len(matching_boms) > 1:
                    print(f"     ⚠️  Found {len(matching_boms)} BOMs with same variant display name")
                    # Take the first one but note the ambiguity
                    bom_v18 = matching_boms[0]
                    match_method = "V14 variant display name vs V18 variant display name (ambiguous - using first match)"
                    print(f"     ⚠️  Using first match (ID: {bom_v18['id']})")
                else:
                    print(f"     ❌ No match")
            
            if not bom_v18:
                print(f"  ❌ NO CORRESPONDING BOM FOUND IN V18 after all matching attempts!")
                mismatches_found += 1
                continue
            
            print(f"  ✅ Found corresponding BOM in V18 (ID: {bom_v18['id']}, Code: '{bom_v18['code']}') via {match_method}")
            
            # Get BOM line components from V18 (from pre-fetched data)
            components_v18 = {}
            bom_lines_v18 = bom_lines_v18_dict.get(bom_v18['id'], [])
            
            if bom_lines_v18:
                print(f"\n  [V18] Components ({len(bom_lines_v18)} items):")
                for line in bom_lines_v18:
                    component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                    component_code = line['product_code']
                    component_qty = line['product_qty']
                    components_v18[component_code] = {'name': component_name, 'qty': component_qty}
                    print(f"    - {component_name} (Code: {component_code}, Qty: {component_qty})")
            else:
                print(f"\n  [V18] Components: None")
            
            # Compare components
            print(f"\n  🔍 COMPARISON RESULTS:")
            has_mismatch = False
            
            # Check for missing components in V18
            for component_code, component_data in components_v14.items():
                if component_code not in components_v18:
                    print(f"    ❌ MISSING IN V18: {component_data['name']} (Code: {component_code}, Qty in V14: {component_data['qty']})")
                    has_mismatch = True
                elif components_v18[component_code]['qty'] != component_data['qty']:
                    print(f"    ⚠️  QUANTITY MISMATCH: {component_data['name']} (Code: {component_code})")
                    print(f"       V14 Qty: {component_data['qty']}")
                    print(f"       V18 Qty: {components_v18[component_code]['qty']}")
                    has_mismatch = True
            
            # Check for extra components in V18
            for component_code, component_data in components_v18.items():
                if component_code not in components_v14:
                    print(f"    ➕ EXTRA IN V18: {component_data['name']} (Code: {component_code}, Qty: {component_data['qty']})")
                    has_mismatch = True
            
            if not has_mismatch:
                print(f"    ✅ All components match!")
            else:
                mismatches_found += 1
    
    print("\n" + "=" * 80)
    print(f"BOM comparison complete!")
    print(f"Total BOMs checked: {len(bom_ids_v14) if bom_ids_v14 else 0}")
    print(f"BOMs with mismatches: {mismatches_found}")
    print("=" * 80)
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


Comparing BOMs between Odoo V14 and V18

[V14] Authenticating...
✅ V14 Authenticated! UID: 28

[V18] Authenticating...
✅ V18 Authenticated! UID: 2

Checking specific categories: [4, 8, 10, 57, 82]

Categories to check:
  - Category ID 4: 0. Presets / 1. Presets zonder PP
  - Category ID 8: 0. Presets / 2. Presets met PP
  - Category ID 10: 0. Presets / 3. Losse verkopen
  - Category ID 57: 0. Presets / 4. Losse PP
  - Category ID 82: 0. Presets / Fixed margin

[V14] Fetching product templates in these categories...
Found 28 product templates

[V14] Searching for BOMs with is_custom_made = False (including inactive)...

[V14] Found 250 BOMs with is_custom_made = False (including inactive)

Comparing BOMs between V14 and V18:
--------------------------------------------------------------------------------

BOM ID (V14): 49
  Reference: Los glas (Blinkend glas)
  Display Name: Los glas (Blinkend glas): Los glas
  Product Template: Los glas
  Product Variant: [P10P] Los glas (Blinkend glas

In [22]:
# Sync BOMs from Odoo V14 to V18 by deleting and recreating them

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Get environment variables for V14
JUSTFRAMEIT_ODOO_V14_URL = os.getenv('JUSTFRAMEIT_ODOO_V14_URL')
JUSTFRAMEIT_ODOO_V14_DB = os.getenv('JUSTFRAMEIT_ODOO_V14_DB')
JUSTFRAMEIT_ODOO_V14_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_V14_USERNAME')
JUSTFRAMEIT_ODOO_V14_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_V14_API_KEY')

# Get environment variables for V18
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL_TEST')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB_TEST')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

print("=" * 80)
print("Syncing BOMs from Odoo V14 to V18")
print("=" * 80)

try:
    # Authenticate to V14
    print("\n[V14] Authenticating...")
    common_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/common", allow_none=True)
    uid_v14 = common_v14.authenticate(JUSTFRAMEIT_ODOO_V14_DB, JUSTFRAMEIT_ODOO_V14_USERNAME, JUSTFRAMEIT_ODOO_V14_API_KEY, {})
    print(f"✅ V14 Authenticated! UID: {uid_v14}")
    
    # Authenticate to V18
    print("\n[V18] Authenticating...")
    common_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
    uid_v18 = common_v18.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
    print(f"✅ V18 Authenticated! UID: {uid_v18}\n")
    
    # Create models proxies
    models_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/object", allow_none=True)
    models_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)
    
    # Define specific category IDs to check
    target_category_ids = [4, 8, 10, 57, 82]
    #target_category_ids = [10] #for testing
    
    print(f"Target categories: {target_category_ids}\n")
    
    # Get category details from V14 (single call)
    categories = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.category', 'read',
        [target_category_ids],
        {'fields': ['display_name']})
    
    print("Categories to sync:")
    for cat in categories:
        print(f"  - Category ID {cat['id']}: {cat['display_name']}")
    print()
    
    # Get all product templates in these categories from V14 (single call)
    print("[V14] Fetching product templates in these categories...")
    product_template_ids = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.template', 'search',
        [[['categ_id', 'in', target_category_ids]]])
    
    print(f"Found {len(product_template_ids)} product templates\n")
    
    # Search for BOMs with is_custom_made = False in V14 (single call)
    print("[V14] Searching for BOMs with is_custom_made = False...")
    print("=" * 80)
    
    bom_ids_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'mrp.bom', 'search',
        [[['product_tmpl_id', 'in', product_template_ids], ['is_custom_made', '=', False]]])
    
    print(f"\n[V14] Found {len(bom_ids_v14)} BOMs with is_custom_made = False\n")
    
    if not bom_ids_v14:
        print("No BOMs found with is_custom_made = False in V14")
    else:
        # Get ALL BOM details from V14 in a single call
        boms_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
            'mrp.bom', 'read',
            [bom_ids_v14],
            {'fields': ['id', 'code', 'display_name', 'product_tmpl_id', 'product_id', 'is_custom_made', 'bom_line_ids', 'product_qty']})
        
        # Get ALL BOM lines from V14 in a single call
        all_bom_line_ids_v14 = []
        for bom in boms_v14:
            if bom['bom_line_ids']:
                all_bom_line_ids_v14.extend(bom['bom_line_ids'])
        
        bom_lines_v14_dict = {}
        if all_bom_line_ids_v14:
            all_bom_lines_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'mrp.bom.line', 'read',
                [all_bom_line_ids_v14],
                {'fields': ['id', 'product_id', 'product_qty', 'bom_id']})
            
            # Get all product IDs from BOM lines to fetch product codes
            product_ids_v14 = [line['product_id'][0] for line in all_bom_lines_v14 if line['product_id']]
            
            # Fetch product codes for all products in V14
            products_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'product.product', 'read',
                [product_ids_v14],
                {'fields': ['id', 'product_code', 'display_name']})
            
            # Create a mapping of product_id to product data
            product_data_map_v14 = {p['id']: {'code': p['product_code'], 'name': p['display_name']} for p in products_v14}
            
            # Organize by bom_id for easy lookup
            for line in all_bom_lines_v14:
                bom_id = line['bom_id'][0] if line['bom_id'] else None
                if bom_id not in bom_lines_v14_dict:
                    bom_lines_v14_dict[bom_id] = []
                # Add product_code to the line data
                product_id = line['product_id'][0] if line['product_id'] else None
                product_data = product_data_map_v14.get(product_id, {'code': 'N/A', 'name': 'N/A'})
                line['product_code'] = product_data['code']
                line['product_name'] = product_data['name']
                bom_lines_v14_dict[bom_id].append(line)
        
        # Step 1: Delete existing BOMs in V18 for these categories
        print("\n" + "=" * 80)
        print("[V18] STEP 1: Deleting existing BOMs in target categories...")
        print("=" * 80)
        
        # Get product templates in V18 for these categories
        product_template_ids_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
            'product.template', 'search',
            [[['categ_id', 'in', target_category_ids]]])
        
        # Find BOMs for these product templates in V18
        bom_ids_v18_to_delete = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
            'mrp.bom', 'search',
            [[['product_tmpl_id', 'in', product_template_ids_v18]]])
        
        print(f"Found {len(bom_ids_v18_to_delete)} BOMs to delete in V18")
        
        deleted_count = 0
        failed_to_delete = []
        
        for bom_id in bom_ids_v18_to_delete:
            try:
                models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'mrp.bom', 'unlink',
                    [[bom_id]])
                deleted_count += 1
                print(f"  ✅ Deleted BOM ID: {bom_id}")
            except Exception as e:
                failed_to_delete.append({'id': bom_id, 'error': str(e)})
                print(f"  ❌ Failed to delete BOM ID {bom_id}: {e}")
        
        print(f"\nDeleted {deleted_count} BOMs successfully")
        if failed_to_delete:
            print(f"Failed to delete {len(failed_to_delete)} BOMs")
        
        # Step 2: Create product code mapping in V18
        print("\n" + "=" * 80)
        print("[V18] STEP 2: Building product code mapping...")
        print("=" * 80)
        
        # Get all products in V18 with their codes
        all_products_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
            'product.product', 'search_read',
            [[]],
            {'fields': ['id', 'x_studio_product_code', 'display_name', 'product_tmpl_id']})
        
        # Create mapping: product_code -> product_id in V18
        product_code_to_id_v18 = {}
        for p in all_products_v18:
            if p['x_studio_product_code']:
                product_code_to_id_v18[p['x_studio_product_code']] = {
                    'id': p['id'],
                    'name': p['display_name'],
                    'tmpl_id': p['product_tmpl_id'][0] if p['product_tmpl_id'] else None
                }
        
        print(f"Mapped {len(product_code_to_id_v18)} products by code in V18")
        
        # Step 3: Recreate BOMs in V18
        print("\n" + "=" * 80)
        print("[V18] STEP 3: Recreating BOMs from V14...")
        print("=" * 80)
        
        created_count = 0
        failed_to_create = []
        created_products = []
        
        for bom_v14 in boms_v14:
            print(f"\n{'=' * 80}")
            print(f"Processing BOM ID (V14): {bom_v14['id']}")
            print(f"  Reference: {bom_v14['code']}")
            print(f"  Display Name: {bom_v14['display_name']}")
            
            # Get product template ID in V18
            product_tmpl_id_v14 = bom_v14['product_tmpl_id'][0] if bom_v14['product_tmpl_id'] else None
            
            if not product_tmpl_id_v14:
                print(f"  ❌ No product template in V14, skipping")
                failed_to_create.append({'bom_id': bom_v14['id'], 'error': 'No product template'})
                continue
            
            # Get product template details from V14
            product_tmpl_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'product.template', 'read',
                [[product_tmpl_id_v14]],
                {'fields': ['default_code', 'name']})
            
            if not product_tmpl_v14:
                print(f"  ❌ Could not read product template from V14, skipping")
                failed_to_create.append({'bom_id': bom_v14['id'], 'error': 'Could not read product template'})
                continue
            
            product_code_v14 = product_tmpl_v14[0]['default_code']
            
            # Find corresponding product template in V18 by code
            product_tmpl_v18_ids = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'product.template', 'search',
                [[['default_code', '=', product_code_v14]]])
            
            if not product_tmpl_v18_ids:
                print(f"  ❌ Product template not found in V18 (code: {product_code_v14}), skipping")
                failed_to_create.append({'bom_id': bom_v14['id'], 'error': f'Product template not found in V18 (code: {product_code_v14})'})
                continue
            
            product_tmpl_id_v18 = product_tmpl_v18_ids[0]
            
            # Handle product variant if specified
            product_id_v18 = False
            if bom_v14['product_id']:
                product_id_v14 = bom_v14['product_id'][0]
                # Get product variant details from V14
                product_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                    'product.product', 'read',
                    [[product_id_v14]],
                    {'fields': ['product_code']})
                
                if product_v14 and product_v14[0]['product_code']:
                    variant_code = product_v14[0]['product_code']
                    # Find variant in V18
                    product_v18_ids = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                        'product.product', 'search',
                        [[['x_studio_product_code', '=', variant_code]]])
                    
                    if product_v18_ids:
                        product_id_v18 = product_v18_ids[0]
            
            # Prepare BOM lines
            bom_lines_v14 = bom_lines_v14_dict.get(bom_v14['id'], [])
            bom_line_vals = []
            
            print(f"  Processing {len(bom_lines_v14)} BOM lines...")
            
            for line in bom_lines_v14:
                component_code = line['product_code']
                component_name = line['product_name']
                component_qty = line['product_qty']
                
                # Find component in V18
                if component_code in product_code_to_id_v18:
                    component_id_v18 = product_code_to_id_v18[component_code]['id']
                    print(f"    ✅ Found component: {component_name} (Code: {component_code})")
                else:
                    # Component doesn't exist, create it
                    print(f"    ⚠️  Component not found in V18: {component_name} (Code: {component_code})")
                    print(f"       Creating component...")
                    
                    try:
                        # Get full product details from V14
                        product_id_v14 = line['product_id'][0] if line['product_id'] else None
                        product_details_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                            'product.product', 'read',
                            [[product_id_v14]],
                            {'fields': ['name', 'product_code', 'categ_id', 'type', 'uom_id', 'uom_po_id']})
                        
                        if product_details_v14:
                            product_detail = product_details_v14[0]
                            
                            # Create product in V18
                            new_product_vals = {
                                'name': product_detail['name'],
                                'x_studio_product_code': component_code,
                                'type': product_detail.get('type', 'product'),
                            }
                            
                            # Add category if exists in V18
                            if product_detail.get('categ_id'):
                                categ_id_v14 = product_detail['categ_id'][0]
                                # Try to find same category in V18
                                categ_v18_ids = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                                    'product.category', 'search',
                                    [[['id', '=', categ_id_v14]]])
                                if categ_v18_ids:
                                    new_product_vals['categ_id'] = categ_v18_ids[0]
                            
                            component_id_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                                'product.product', 'create',
                                [new_product_vals])
                            
                            # Update mapping
                            product_code_to_id_v18[component_code] = {
                                'id': component_id_v18,
                                'name': product_detail['name'],
                                'tmpl_id': None
                            }
                            
                            created_products.append({
                                'id': component_id_v18,
                                'code': component_code,
                                'name': product_detail['name']
                            })
                            
                            print(f"       ✅ Created component ID: {component_id_v18}")
                        else:
                            print(f"       ❌ Could not read product details from V14, skipping component")
                            continue
                    except Exception as e:
                        print(f"       ❌ Failed to create component: {e}")
                        continue
                
                # Add BOM line
                bom_line_vals.append((0, 0, {
                    'product_id': component_id_v18,
                    'product_qty': component_qty,
                }))
            
            # Create BOM in V18
            try:
                bom_reference = f"{bom_v14['code']} (V14 BOM ID: {bom_v14['id']})" if bom_v14['code'] else f"BOM (V14 BOM ID: {bom_v14['id']})"
                
                bom_vals = {
                    'product_tmpl_id': product_tmpl_id_v18,
                    'code': bom_reference,
                    'product_qty': bom_v14.get('product_qty', 1.0),
                    'bom_line_ids': bom_line_vals,
                }
                
                if product_id_v18:
                    bom_vals['product_id'] = product_id_v18
                
                new_bom_id = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'mrp.bom', 'create',
                    [bom_vals])
                
                created_count += 1
                print(f"  ✅ Created BOM in V18 with ID: {new_bom_id}")
                print(f"     Reference: {bom_reference}")
                
            except Exception as e:
                print(f"  ❌ Failed to create BOM: {e}")
                failed_to_create.append({'bom_id': bom_v14['id'], 'error': str(e)})
        
        # Summary
        print("\n" + "=" * 80)
        print("SYNC SUMMARY")
        print("=" * 80)
        print(f"Total BOMs in V14: {len(boms_v14)}")
        print(f"BOMs deleted in V18: {deleted_count}")
        print(f"BOMs created in V18: {created_count}")
        print(f"Failed to create: {len(failed_to_create)}")
        print(f"Products created: {len(created_products)}")
        
        if created_products:
            print("\nProducts created in V18:")
            for prod in created_products:
                print(f"  - {prod['name']} (Code: {prod['code']}, ID: {prod['id']})")
        
        if failed_to_create:
            print("\nFailed to create BOMs:")
            for fail in failed_to_create:
                print(f"  - BOM ID (V14): {fail['bom_id']}, Error: {fail['error']}")
        
        print("=" * 80)
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


Syncing BOMs from Odoo V14 to V18

[V14] Authenticating...
✅ V14 Authenticated! UID: 28

[V18] Authenticating...
✅ V18 Authenticated! UID: 2

Target categories: [4, 8, 10, 57, 82]

Categories to sync:
  - Category ID 4: 0. Presets / 1. Presets zonder PP
  - Category ID 8: 0. Presets / 2. Presets met PP
  - Category ID 10: 0. Presets / 3. Losse verkopen
  - Category ID 57: 0. Presets / 4. Losse PP
  - Category ID 82: 0. Presets / Fixed margin

[V14] Fetching product templates in these categories...
Found 28 product templates

[V14] Searching for BOMs with is_custom_made = False...

[V14] Found 249 BOMs with is_custom_made = False


[V18] STEP 1: Deleting existing BOMs in target categories...
Found 1 BOMs to delete in V18
  ❌ Failed to delete BOM ID 153: <Fault 2: 'You can not delete a Bill of Material with running manufacturing orders.\nPlease close or cancel it first.'>

Deleted 0 BOMs successfully
Failed to delete 1 BOMs

[V18] STEP 2: Building product code mapping...
Mapped 3229 prod

In [25]:
# Compare BOMs between Odoo V14 and V18 to find mismatches

import os
import xmlrpc.client
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Get environment variables for V14
JUSTFRAMEIT_ODOO_V14_URL = os.getenv('JUSTFRAMEIT_ODOO_V14_URL')
JUSTFRAMEIT_ODOO_V14_DB = os.getenv('JUSTFRAMEIT_ODOO_V14_DB')
JUSTFRAMEIT_ODOO_V14_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_V14_USERNAME')
JUSTFRAMEIT_ODOO_V14_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_V14_API_KEY')

# Get environment variables for V18
JUSTFRAMEIT_ODOO_URL = os.getenv('JUSTFRAMEIT_ODOO_URL_TEST')
JUSTFRAMEIT_ODOO_DB = os.getenv('JUSTFRAMEIT_ODOO_DB_TEST')
JUSTFRAMEIT_ODOO_USERNAME = os.getenv('JUSTFRAMEIT_ODOO_USERNAME')
JUSTFRAMEIT_ODOO_API_KEY = os.getenv('JUSTFRAMEIT_ODOO_API_KEY')

print("=" * 80)
print("Comparing BOMs between Odoo V14 and V18")
print("=" * 80)

try:
    # Authenticate to V14
    print("\n[V14] Authenticating...")
    common_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/common", allow_none=True)
    uid_v14 = common_v14.authenticate(JUSTFRAMEIT_ODOO_V14_DB, JUSTFRAMEIT_ODOO_V14_USERNAME, JUSTFRAMEIT_ODOO_V14_API_KEY, {})
    print(f"✅ V14 Authenticated! UID: {uid_v14}")
    
    # Authenticate to V18
    print("\n[V18] Authenticating...")
    common_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/common", allow_none=True)
    uid_v18 = common_v18.authenticate(JUSTFRAMEIT_ODOO_DB, JUSTFRAMEIT_ODOO_USERNAME, JUSTFRAMEIT_ODOO_API_KEY, {})
    print(f"✅ V18 Authenticated! UID: {uid_v18}\n")
    
    # Create models proxies
    models_v14 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_V14_URL}/xmlrpc/2/object", allow_none=True)
    models_v18 = xmlrpc.client.ServerProxy(f"{JUSTFRAMEIT_ODOO_URL}/xmlrpc/2/object", allow_none=True)
    
    # Define specific category IDs to check
    target_category_ids = [4, 8, 10, 57, 82]
    #target_category_ids = [10] #for testing
    
    print(f"Checking specific categories: {target_category_ids}\n")
    
    # Get category details from V14 (single call)
    categories = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.category', 'read',
        [target_category_ids],
        {'fields': ['display_name']})
    
    print("Categories to check:")
    for cat in categories:
        print(f"  - Category ID {cat['id']}: {cat['display_name']}")
    print()
    
    # Get all product templates in these categories from V14 (single call)
    print("[V14] Fetching product templates in these categories...")
    product_template_ids = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'product.template', 'search',
        [[['categ_id', 'in', target_category_ids]]])
    
    print(f"Found {len(product_template_ids)} product templates\n")
    
    # Search for BOMs with is_custom_made = False in V14 (single call) - including inactive BOMs
    print("[V14] Searching for BOMs with is_custom_made = False (including inactive BOMs)...")
    print("=" * 80)
    
    bom_ids_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
        'mrp.bom', 'search',
        [[['product_tmpl_id', 'in', product_template_ids], ['is_custom_made', '=', False]]],
        {'context': {'active_test': False}})
    
    print(f"\n[V14] Found {len(bom_ids_v14)} BOMs with is_custom_made = False (including inactive)\n")
    
    if not bom_ids_v14:
        print("No BOMs found with is_custom_made = False in V14")
    else:
        # Get ALL BOM details from V14 in a single call (including active field)
        boms_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
            'mrp.bom', 'read',
            [bom_ids_v14],
            {'fields': ['id', 'code', 'display_name', 'product_tmpl_id', 'product_id', 'is_custom_made', 'bom_line_ids', 'active']})
        
        # Get ALL BOM lines from V14 in a single call
        all_bom_line_ids_v14 = []
        for bom in boms_v14:
            if bom['bom_line_ids']:
                all_bom_line_ids_v14.extend(bom['bom_line_ids'])
        
        bom_lines_v14_dict = {}
        if all_bom_line_ids_v14:
            all_bom_lines_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'mrp.bom.line', 'read',
                [all_bom_line_ids_v14],
                {'fields': ['id', 'product_id', 'product_qty', 'bom_id']})
            
            # Get all product IDs from BOM lines to fetch product codes
            product_ids_v14 = [line['product_id'][0] for line in all_bom_lines_v14 if line['product_id']]
            
            # Fetch product codes for all products in V14
            products_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'product.product', 'read',
                [product_ids_v14],
                {'fields': ['id', 'product_code']})
            
            # Create a mapping of product_id to product_code
            product_code_map_v14 = {p['id']: p['product_code'] for p in products_v14}
            
            # Organize by bom_id for easy lookup
            for line in all_bom_lines_v14:
                bom_id = line['bom_id'][0] if line['bom_id'] else None
                if bom_id not in bom_lines_v14_dict:
                    bom_lines_v14_dict[bom_id] = []
                # Add product_code to the line data
                product_id = line['product_id'][0] if line['product_id'] else None
                line['product_code'] = product_code_map_v14.get(product_id, 'N/A')
                bom_lines_v14_dict[bom_id].append(line)
        
        # Collect all BOM codes to search in V18
        bom_codes_v14 = [bom['code'] for bom in boms_v14 if bom['code']]
        
        # Search for ALL corresponding BOMs in V18 in a single call (including inactive)
        bom_ids_v18 = []
        if bom_codes_v14:
            bom_ids_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'mrp.bom', 'search',
                [[['code', 'in', bom_codes_v14]]],
                {'context': {'active_test': False}})
        
        # Get ALL BOM details from V18 in a single call (including active field)
        boms_v18_dict = {}
        if bom_ids_v18:
            boms_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'mrp.bom', 'read',
                [bom_ids_v18],
                {'fields': ['id', 'code', 'display_name', 'product_tmpl_id', 'product_id', 'bom_line_ids', 'active']})
            
            # Organize by code for easy lookup (handle duplicates by storing as list)
            for bom in boms_v18:
                if bom['code']:
                    if bom['code'] not in boms_v18_dict:
                        boms_v18_dict[bom['code']] = []
                    boms_v18_dict[bom['code']].append(bom)
            
            # Get ALL BOM lines from V18 in a single call
            all_bom_line_ids_v18 = []
            for bom in boms_v18:
                if bom['bom_line_ids']:
                    all_bom_line_ids_v18.extend(bom['bom_line_ids'])
            
            bom_lines_v18_dict = {}
            if all_bom_line_ids_v18:
                all_bom_lines_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'mrp.bom.line', 'read',
                    [all_bom_line_ids_v18],
                    {'fields': ['id', 'product_id', 'product_qty', 'bom_id']})
                
                # Get all product IDs from BOM lines to fetch product codes
                product_ids_v18 = [line['product_id'][0] for line in all_bom_lines_v18 if line['product_id']]
                
                # Fetch product codes for all products in V18
                products_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                    'product.product', 'read',
                    [product_ids_v18],
                    {'fields': ['id', 'x_studio_product_code']})
                
                # Create a mapping of product_id to x_studio_product_code
                product_code_map_v18 = {p['id']: p['x_studio_product_code'] for p in products_v18}
                
                # Organize by bom_id for easy lookup
                for line in all_bom_lines_v18:
                    bom_id = line['bom_id'][0] if line['bom_id'] else None
                    if bom_id not in bom_lines_v18_dict:
                        bom_lines_v18_dict[bom_id] = []
                    # Add product_code to the line data
                    product_id = line['product_id'][0] if line['product_id'] else None
                    line['product_code'] = product_code_map_v18.get(product_id, 'N/A')
                    bom_lines_v18_dict[bom_id].append(line)
        
        # Fetch product codes for all product_ids in V14 BOMs for fallback matching
        all_product_ids_v14 = [bom['product_id'][0] for bom in boms_v14 if bom['product_id']]
        product_codes_v14_map = {}
        product_variants_v14_map = {}
        if all_product_ids_v14:
            products_for_boms_v14 = models_v14.execute_kw(JUSTFRAMEIT_ODOO_V14_DB, uid_v14, JUSTFRAMEIT_ODOO_V14_API_KEY,
                'product.product', 'read',
                [all_product_ids_v14],
                {'fields': ['id', 'product_code', 'display_name']})
            product_codes_v14_map = {p['id']: p['product_code'] for p in products_for_boms_v14}
            product_variants_v14_map = {p['id']: p['display_name'] for p in products_for_boms_v14}
        
        # Fetch product variants for all product_ids in V18 BOMs for fallback matching
        all_product_ids_v18 = []
        for bom_list in boms_v18_dict.values():
            for bom in bom_list:
                if bom['product_id']:
                    all_product_ids_v18.append(bom['product_id'][0])
        
        product_variants_v18_map = {}
        if all_product_ids_v18:
            products_for_boms_v18 = models_v18.execute_kw(JUSTFRAMEIT_ODOO_DB, uid_v18, JUSTFRAMEIT_ODOO_API_KEY,
                'product.product', 'read',
                [all_product_ids_v18],
                {'fields': ['id', 'x_studio_product_code', 'display_name']})
            product_variants_v18_map = {p['id']: {'code': p['x_studio_product_code'], 'name': p['display_name']} for p in products_for_boms_v18}
        
        print("Comparing BOMs between V14 and V18:")
        print("-" * 80)
        
        mismatches_found = 0
        
        for bom_v14 in boms_v14:
            product_tmpl_name = bom_v14['product_tmpl_id'][1] if bom_v14['product_tmpl_id'] else 'N/A'
            product_variant_name = bom_v14['product_id'][1] if bom_v14['product_id'] else 'N/A (Template BOM)'
            bom_reference = bom_v14['code'] if bom_v14['code'] else 'No reference'
            bom_display_name = bom_v14['display_name'] if bom_v14['display_name'] else 'No display name'
            bom_active_v14 = bom_v14.get('active', True)
            
            print(f"\n{'=' * 80}")
            print(f"BOM ID (V14): {bom_v14['id']}")
            print(f"  Reference: {bom_reference}")
            print(f"  Display Name: {bom_display_name}")
            print(f"  Product Template: {product_tmpl_name}")
            print(f"  Product Variant: {product_variant_name}")
            print(f"  is_custom_made (V14): {bom_v14['is_custom_made']}")
            print(f"  active (V14): {bom_active_v14}")
            
            # Get BOM line components from V14 (from pre-fetched data)
            components_v14 = {}
            bom_lines = bom_lines_v14_dict.get(bom_v14['id'], [])
            
            if bom_lines:
                print(f"\n  [V14] Components ({len(bom_lines)} items):")
                for line in bom_lines:
                    component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                    component_code = line['product_code']
                    component_qty = line['product_qty']
                    components_v14[component_code] = {'name': component_name, 'qty': component_qty}
                    print(f"    - {component_name} (Code: {component_code}, Qty: {component_qty})")
            else:
                print(f"\n  [V14] Components: None")
            
            # Find corresponding BOM in V18
            print(f"\n  [V18] Searching for corresponding BOM...")
            
            bom_v18 = None
            matching_method = None
            
            # Try matching by BOM reference (code) first
            if bom_reference and bom_reference != 'No reference':
                print(f"     Trying to match by BOM reference (code): {bom_reference}")
                
                matching_boms = boms_v18_dict.get(bom_reference, [])
                
                if len(matching_boms) == 1:
                    # Unique match by code
                    bom_v18 = matching_boms[0]
                    matching_method = "by BOM reference (code)"
                    print(f"  ✅ Found unique BOM in V18 by reference")
                elif len(matching_boms) > 1:
                    # Multiple BOMs with same code - need to use product_id fallback
                    print(f"  ⚠️  Found {len(matching_boms)} BOMs with same reference in V18, using product_id fallback")
                    
                    # Get product_code from V14 product_id
                    product_id_v14 = bom_v14['product_id'][0] if bom_v14['product_id'] else None
                    if product_id_v14:
                        product_code_v14 = product_codes_v14_map.get(product_id_v14)
                        product_variant_v14 = product_variants_v14_map.get(product_id_v14)
                        if product_code_v14:
                            print(f"     V14 product_code: {product_code_v14}")
                            print(f"     V14 product_variant: {product_variant_v14}")
                            
                            # Try to match by product_code in V18
                            for candidate_bom in matching_boms:
                                if candidate_bom['product_id']:
                                    product_id_v18 = candidate_bom['product_id'][0]
                                    product_info_v18 = product_variants_v18_map.get(product_id_v18, {})
                                    product_code_v18 = product_info_v18.get('code')
                                    product_variant_v18 = product_info_v18.get('name')
                                    
                                    print(f"       Checking candidate: product_code={product_code_v18}, variant={product_variant_v18}")
                                    
                                    if product_code_v18 == product_code_v14:
                                        bom_v18 = candidate_bom
                                        matching_method = "by BOM reference + product_code fallback"
                                        print(f"  ✅ Matched by product_code: {product_code_v18}")
                                        break
                            
                            # If still not matched, try comparing product variants (display_name)
                            if not bom_v18 and product_variant_v14:
                                print(f"     Trying to match by product variant display_name")
                                for candidate_bom in matching_boms:
                                    if candidate_bom['product_id']:
                                        product_id_v18 = candidate_bom['product_id'][0]
                                        product_info_v18 = product_variants_v18_map.get(product_id_v18, {})
                                        product_variant_v18 = product_info_v18.get('name')
                                        
                                        if product_variant_v18 == product_variant_v14:
                                            bom_v18 = candidate_bom
                                            matching_method = "by BOM reference + product variant display_name fallback"
                                            print(f"  ✅ Matched by product variant: {product_variant_v18}")
                                            break
                            
                            if not bom_v18:
                                print(f"  ❌ Could not match any of the {len(matching_boms)} BOMs by product_code or product variant")
                        else:
                            print(f"  ❌ Could not get product_code from V14 product_id")
                    else:
                        print(f"  ❌ No product_id in V14 BOM for fallback matching")
            
            # Fallback: try matching by product_code if no reference or reference didn't match
            if not bom_v18:
                print(f"     Trying fallback: matching by product_code")
                
                product_id_v14 = bom_v14['product_id'][0] if bom_v14['product_id'] else None
                if product_id_v14:
                    product_code_v14 = product_codes_v14_map.get(product_id_v14)
                    product_variant_v14 = product_variants_v14_map.get(product_id_v14)
                    if product_code_v14:
                        print(f"     V14 product_code: {product_code_v14}")
                        print(f"     V14 product_variant: {product_variant_v14}")
                        
                        # Search through all V18 BOMs for matching product_code
                        for code, bom_list in boms_v18_dict.items():
                            for candidate_bom in bom_list:
                                if candidate_bom['product_id']:
                                    product_id_v18 = candidate_bom['product_id'][0]
                                    product_info_v18 = product_variants_v18_map.get(product_id_v18, {})
                                    product_code_v18 = product_info_v18.get('code')
                                    if product_code_v18 == product_code_v14:
                                        bom_v18 = candidate_bom
                                        matching_method = "by product_code fallback"
                                        print(f"  ✅ Found BOM in V18 by product_code: {product_code_v18}")
                                        break
                            if bom_v18:
                                break
                        
                        # If still not matched, try comparing product variants (display_name)
                        if not bom_v18 and product_variant_v14:
                            print(f"     Trying to match by product variant display_name")
                            for code, bom_list in boms_v18_dict.items():
                                for candidate_bom in bom_list:
                                    if candidate_bom['product_id']:
                                        product_id_v18 = candidate_bom['product_id'][0]
                                        product_info_v18 = product_variants_v18_map.get(product_id_v18, {})
                                        product_variant_v18 = product_info_v18.get('name')
                                        
                                        if product_variant_v18 == product_variant_v14:
                                            bom_v18 = candidate_bom
                                            matching_method = "by product variant display_name fallback"
                                            print(f"  ✅ Found BOM in V18 by product variant: {product_variant_v18}")
                                            break
                                if bom_v18:
                                    break
                    else:
                        print(f"  ❌ Could not get product_code from V14 product_id")
                else:
                    print(f"  ❌ No product_id in V14 BOM for fallback matching")
            
            if not bom_v18:
                print(f"  ❌ NO CORRESPONDING BOM FOUND IN V18!")
                mismatches_found += 1
                continue
            
            bom_active_v18 = bom_v18.get('active', True)
            print(f"  ✅ Found corresponding BOM in V18 (ID: {bom_v18['id']}, Code: '{bom_v18['code']}', active: {bom_active_v18}, matched {matching_method})")
            
            # Get BOM line components from V18 (from pre-fetched data)
            components_v18 = {}
            bom_lines_v18 = bom_lines_v18_dict.get(bom_v18['id'], [])
            
            if bom_lines_v18:
                print(f"\n  [V18] Components ({len(bom_lines_v18)} items):")
                for line in bom_lines_v18:
                    component_name = line['product_id'][1] if line['product_id'] else 'N/A'
                    component_code = line['product_code']
                    component_qty = line['product_qty']
                    components_v18[component_code] = {'name': component_name, 'qty': component_qty}
                    print(f"    - {component_name} (Code: {component_code}, Qty: {component_qty})")
            else:
                print(f"\n  [V18] Components: None")
            
            # Compare components
            print(f"\n  🔍 COMPARISON RESULTS:")
            has_mismatch = False
            
            # Check for missing components in V18
            for component_code, component_data in components_v14.items():
                if component_code not in components_v18:
                    print(f"    ❌ MISSING IN V18: {component_data['name']} (Code: {component_code}, Qty in V14: {component_data['qty']})")
                    has_mismatch = True
                elif components_v18[component_code]['qty'] != component_data['qty']:
                    print(f"    ⚠️  QUANTITY MISMATCH: {component_data['name']} (Code: {component_code})")
                    print(f"       V14 Qty: {component_data['qty']}")
                    print(f"       V18 Qty: {components_v18[component_code]['qty']}")
                    has_mismatch = True
            
            # Check for extra components in V18
            for component_code, component_data in components_v18.items():
                if component_code not in components_v14:
                    print(f"    ➕ EXTRA IN V18: {component_data['name']} (Code: {component_code}, Qty: {component_data['qty']})")
                    has_mismatch = True
            
            if not has_mismatch:
                print(f"    ✅ All components match!")
            else:
                mismatches_found += 1
    
    print("\n" + "=" * 80)
    print(f"BOM comparison complete!")
    print(f"Total BOMs checked: {len(bom_ids_v14) if bom_ids_v14 else 0}")
    print(f"BOMs with mismatches: {mismatches_found}")
    print("=" * 80)
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


Comparing BOMs between Odoo V14 and V18

[V14] Authenticating...
✅ V14 Authenticated! UID: 28

[V18] Authenticating...
✅ V18 Authenticated! UID: 2

Checking specific categories: [4, 8, 10, 57, 82]

Categories to check:
  - Category ID 4: 0. Presets / 1. Presets zonder PP
  - Category ID 8: 0. Presets / 2. Presets met PP
  - Category ID 10: 0. Presets / 3. Losse verkopen
  - Category ID 57: 0. Presets / 4. Losse PP
  - Category ID 82: 0. Presets / Fixed margin

[V14] Fetching product templates in these categories...
Found 28 product templates

[V14] Searching for BOMs with is_custom_made = False (including inactive BOMs)...

[V14] Found 250 BOMs with is_custom_made = False (including inactive)

Comparing BOMs between V14 and V18:
--------------------------------------------------------------------------------

BOM ID (V14): 49
  Reference: Los glas (Blinkend glas)
  Display Name: Los glas (Blinkend glas): Los glas
  Product Template: Los glas
  Product Variant: [P10P] Los glas (Blinkend